In [37]:
!feast remove

/home/asok/anaconda3/envs/mlops/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Usage: feast [OPTIONS] COMMAND [ARGS]...
Try 'feast --help' for help.

Error: No such command 'remove'.


In [39]:
# Using feast to create feature store for the repo
#install feat

!pip install feast==0.11.0
!pip freeze | grep feast

  Using cached feast-0.11.0-py3-none-any.whl (190 kB)
  Using cached fastavro-1.4.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.7 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Attempting uninstall: fastavro
    Found existing installation: fastavro 0.22.13
    Uninstalling fastavro-0.22.13:
      Successfully uninstalled fastavro-0.22.13
  Attempting uninstall: tenacity
    Found existing installation: tenacity 6.2.0
    Uninstalling tenacity-6.2.0:
      Successfully uninstalled tenacity-6.2.0
  Attempting uninstall: feast
    Found existing installation: feast 0.10.5
    Uninstalling feast-0.10.5:
      Successfully uninstalled feast-0.10.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-airflow 2.1.2 requires tenacity~=6.2.0, but you have tenacity 8.0.1 which is incompatible.
feast==0.11.0


In [85]:
#%%bash
#cd ../
#feast init --minimal --template local features
#cd features
#touch features.py

In [40]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.insert(0, '/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core')
print(sys.path)
import config

['/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/ML OPS/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core', '/c//toolbox/mlops/End_To_End_

In [41]:
import sys

sys.path

['/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/home/asok/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/ML OPS/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c/toolbox/mlops/End_To_End_Model_Deployment_MLOps/core',
 '/c//toolbox/m

In [62]:
# Loading features to data frame
feature_path = Path(config.DATA_DIR, "credit_card_transactions.csv")

features = pd.read_csv(feature_path)

features['created_time'] = features['created_time'].astype({'created_time': 'datetime64[ms]'})

#features['created_time'] = pd.to_datetime(features['created_time'], unit='ms')

print(features.head())

features.info()

   Unnamed: 0  customer_id  Time        V1        V2        V3        V4  \
0           0            0   0.0 -1.359807 -0.072781  2.536347  1.378155   
1           1            1   0.0  1.191857  0.266151  0.166480  0.448154   
2           2            2   1.0 -1.358354 -1.340163  1.773209  0.379780   
3           3            3   1.0 -0.966272 -0.185226  1.792993 -0.863291   
4           4            4   2.0 -1.158233  0.877737  1.548718  0.403034   

         V5  Amount  Class created_time  
0 -0.338321  149.62      0   2020-09-07  
1  0.060018    2.69      0   2020-11-22  
2 -0.503198  378.66      0   2020-10-02  
3 -0.010309  123.50      0   2020-09-06  
4 -0.407193   69.99      0   2020-07-12  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    284807 non-null  int64         
 1   customer_id   284807 n

In [63]:
# Convert to parquet format as Feast only support it thorugh offline

features.to_parquet(
    Path(config.DATA_DIR, "features.parquet"),
    compression=None,
    allow_truncated_timestamps=True
)

/home/asok/anaconda3/envs/mlops/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Feature Store


In [64]:
from feast import Entity, Feature, FeatureView, ValueType
from feast.data_source import FileSource
from google.protobuf.duration_pb2 import Duration
from datetime import datetime

define the location of the data source to feast

In [65]:
START_TIME = "2021-08-05"
feature_source = FileSource(
    path=str(Path(config.DATA_DIR, "features.parquet")),
    event_timestamp_column="created_time"
)

Nextly, define the main entity of feature data points. In our case, each data point has a unique ID.

In [66]:
feature_entity = Entity(
    name="customer_id",
    value_type=ValueType.INT64,
    description="customer id"
)

Finally, create the Feature View that loads specific features of valrious value types from the feature source for a specific period of time

In [67]:
feature_details_view = FeatureView(
    name="credit_card_transactions",
    entities=["customer_id"],
     ttl=Duration(
        seconds=(datetime.today() - datetime.strptime(START_TIME, "%Y-%m-%d")).days * 24 * 60 * 60
    ),
    features=[
        Feature(name="V1", dtype=ValueType.FLOAT),
        Feature(name="V2", dtype=ValueType.FLOAT),
        Feature(name="V3", dtype=ValueType.FLOAT),
        Feature(name="V4", dtype=ValueType.FLOAT),
        Feature(name="V5", dtype=ValueType.FLOAT), 
        Feature(name="Amount", dtype=ValueType.FLOAT),
        Feature(name="Time", dtype=ValueType.FLOAT),
        Feature(name="Class", dtype=ValueType.FLOAT)
    ],
    online=True,
    input=feature_source,
    tags={"transaction": "fradulant_transaction"},
    )

In [128]:
#%%bash
#cd ../features
#feast apply

In [188]:
%%cmd
cd ../features
feast apply

Microsoft Windows [Version 10.0.19043.1110]
(c) Microsoft Corporation. All rights reserved.

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\notebooks>cd ../features

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>feast apply
Registered entity ID
Registered feature view house_price_details
Deploying infrastructure for house_price_details

(mlops) c:\toolbox\ML OPS\End_To_End_Model_Deployment_MLOps\features>

In [68]:
import pandas as pd
from feast import FeatureStore
from datetime import datetime

In [69]:
customer_tran_id = [1, 2, 33]
now = datetime.now()
timestamps = [datetime(now.year, now.month, now.day)] *len(customer_tran_id)
entity_df = pd.DataFrame.from_dict({"customer_id": customer_tran_id, "event_timestamp": timestamps})
entity_df.head()

,customer_id,event_timestamp
0,1,2021-08-05
1,2,2021-08-05
2,33,2021-08-05


In [73]:
#Get historical features
store = FeatureStore(repo_path=Path(config.BASE_DIR, "features"))
training_df = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=[
        "credit_card_transactions:V1", 
        "credit_card_transactions:V2",
        "credit_card_transactions:V3", 
        "credit_card_transactions:V4",
        "credit_card_transactions:V5",
        "credit_card_transactions:Amount",
        "credit_card_transactions:Time",
        "credit_card_transactions:Class",
        ],
).to_df()
training_df.head()

/home/asok/anaconda3/envs/mlops/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,event_timestamp,customer_id,credit_card_transactions__V3,credit_card_transactions__V4,credit_card_transactions__V5,credit_card_transactions__Amount,credit_card_transactions__Time,credit_card_transactions__Class
0,2021-08-05 00:00:00+00:00,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-05 00:00:00+00:00,2,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-05 00:00:00+00:00,33,NaN,NaN,NaN,NaN,NaN,NaN


Materialize - Online Store

In [ ]:
%%bash
cd ../features
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME

In [55]:
# Get online features
store = FeatureStore(repo_path=Path(config.BASE_DIR, "features"))
feature_vector = store.get_online_features(
    feature_refs=["credit_card_transactions:V1", "credit_card_transactions:V2"],
    entity_rows=[{"customer_id": 2}],
).to_dict()
feature_vector

{'customer_id': [2],
 'credit_card_transactions__V1': [-1.35835406159823],
 'credit_card_transactions__V2': [-1.3401630747360902]}